# 🚦 Traffic Manager AI - Full Version
### Vehicle Detection + Traffic Light + License Plate OCR

**Models cần có trên Kaggle:**
- `yolo11m.pt` hoặc `mhiot-vehicle-best-new.pt` (Vehicle)
- `mhiot-dentinhieu-best-new.pt` (Traffic Light)
- `LP_detector.pt` + `LP_ocr.pt` (License Plate)

In [ ]:
# Cell 1: Install Dependencies
!pip install ultralytics python-socketio[client] websocket-client opencv-python-headless --quiet

# Download default vehicle model if not exists
!wget -nc -q https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11m.pt

# Clone YOLOv5 for license plate (if needed)
import os
if not os.path.exists('yolov5'):
    !git clone --depth 1 https://github.com/ultralytics/yolov5.git 2>/dev/null
print("✅ Dependencies ready!")

In [ ]:
# Cell 2: Load All Models
import torch
import sys
from ultralytics import YOLO

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🚀 Loading models on {device}...")

# ========== MODEL PATHS (cập nhật theo file của bạn) ==========
VEHICLE_MODEL = 'yolo11m.pt'  # hoặc 'mhiot-vehicle-best-new.pt'
TRAFFIC_LIGHT_MODEL = 'mhiot-dentinhieu-best-new.pt'
LP_DETECTOR_MODEL = 'LP_detector.pt'
LP_OCR_MODEL = 'LP_ocr.pt'
# ==============================================================

# Load Vehicle Model (YOLOv8/11)
vehicle_model = None
try:
    vehicle_model = YOLO(VEHICLE_MODEL).to(device)
    print(f"✅ Vehicle model loaded: {VEHICLE_MODEL}")
except Exception as e:
    print(f"⚠️ Vehicle model not found: {e}")

# Load Traffic Light Model (YOLOv8/11)
tl_model = None
try:
    tl_model = YOLO(TRAFFIC_LIGHT_MODEL).to(device)
    print(f"✅ Traffic Light model loaded: {TRAFFIC_LIGHT_MODEL}")
except Exception as e:
    print(f"⚠️ Traffic Light model not found: {e}")

# Load License Plate Models (YOLOv5)
lp_detector = None
lp_ocr = None
try:
    sys.path.insert(0, 'yolov5')
    lp_detector = torch.hub.load('yolov5', 'custom', path=LP_DETECTOR_MODEL, source='local', verbose=False)
    lp_ocr = torch.hub.load('yolov5', 'custom', path=LP_OCR_MODEL, source='local', verbose=False)
    lp_detector.to(device)
    lp_ocr.to(device)
    lp_detector.conf = 0.3
    lp_ocr.conf = 0.5
    print(f"✅ License Plate models loaded: {LP_DETECTOR_MODEL}, {LP_OCR_MODEL}")
except Exception as e:
    print(f"⚠️ License Plate models not found: {e}")

print("\n📊 Summary:")
print(f"  Vehicle: {'✅' if vehicle_model else '❌'}")
print(f"  Traffic Light: {'✅' if tl_model else '❌'}")
print(f"  License Plate: {'✅' if lp_detector and lp_ocr else '❌'}")

In [ ]:
# Cell 3: Main Detection Service
import cv2, numpy as np, socketio, base64, time, threading, queue, logging, warnings, math, re, io
from datetime import datetime
from PIL import Image

# ========== SERVER URL ==========
SERVER_URL = 'https://liberal-surrounding-lease-estimates.trycloudflare.com'
# ================================

logging.getLogger('socketio').setLevel(logging.WARNING)
logging.getLogger('engineio').setLevel(logging.WARNING)
warnings.filterwarnings('ignore')

VEHICLE_CLASSES = ['car', 'truck', 'bus', 'motorcycle', 'bicycle']
CONFIDENCE = 0.5

sio = socketio.Client(reconnection=True, reconnection_attempts=0, reconnection_delay=1, 
                      ssl_verify=False, logger=False, engineio_logger=False)

cam_queues = {}
lp_queue = queue.Queue(maxsize=5)
running = True
stats = {'f': 0, 'v': 0, 'tl': 0, 'lp': 0, 'c': 0, 'dc': 0}

def log(msg): print(f"[{datetime.now().strftime('%H:%M:%S')}] {msg}")

# ---------- License Plate Helpers ----------
def read_plate(model, im):
    results = model(im)
    bb_list = results.pandas().xyxy[0].values.tolist()
    if len(bb_list) < 7 or len(bb_list) > 10: return "unknown"
    bb_list.sort(key=lambda x: x[0])
    ys = [b[1] for b in bb_list]
    y_mean = sum(ys) / len(ys)
    if max(ys) - min(ys) > im.shape[0] * 0.3:
        line1 = sorted([b for b in bb_list if b[1] < y_mean], key=lambda x: x[0])
        line2 = sorted([b for b in bb_list if b[1] >= y_mean], key=lambda x: x[0])
        return ''.join([str(b[-1]) for b in line1]) + '-' + ''.join([str(b[-1]) for b in line2])
    return ''.join([str(b[-1]) for b in bb_list])

def deskew(img):
    try:
        h, w = img.shape[:2]
        blur = cv2.medianBlur(img, 3)
        edges = cv2.Canny(blur, 30, 100)
        lines = cv2.HoughLinesP(edges, 1, math.pi/180, 30, minLineLength=w/2, maxLineGap=h/3)
        if lines is None: return img
        angle = np.mean([np.arctan2(l[0][3]-l[0][1], l[0][2]-l[0][0]) for l in lines[:5]])
        M = cv2.getRotationMatrix2D((w/2, h/2), angle*180/math.pi, 1)
        return cv2.warpAffine(img, M, (w, h))
    except: return img

# ---------- Processing Threads ----------
def process_camera(cid):
    log(f"📹 Cam {cid[-4:]} started")
    while running:
        try:
            data = cam_queues[cid].get(timeout=0.5)
            if data is None: continue
            f, c, i, t, line_y = data
            h, w = f.shape[:2]
            dets = []
            
            # 1. Vehicle Detection
            if vehicle_model:
                for r in vehicle_model.track(f, persist=True, verbose=False):
                    for b in r.boxes:
                        cls = vehicle_model.names[int(b.cls[0])]
                        if cls not in VEHICLE_CLASSES or float(b.conf[0]) < CONFIDENCE: continue
                        x1,y1,x2,y2 = map(int, b.xyxy[0])
                        det = {'class':cls,'type':'vehicle','confidence':float(b.conf[0]),
                               'bbox':{'x1':x1/w,'y1':y1/h,'x2':x2/w,'y2':y2/h}}
                        if hasattr(b,'id') and b.id is not None:
                            det['id'] = int(b.id[0])
                        dets.append(det)
                        stats['v'] += 1
            
            # Emit car_detected
            if dets and sio.connected:
                sio.emit('car_detected', {'camera_id':c,'image_id':i,'detections':dets,
                         'track_line_y':line_y,'created_at':t})
                log(f"✅ {c[-4:]}: {len(dets)} vehicles")
            
            # 2. Traffic Light Detection
            if tl_model:
                tl_dets = []
                for r in tl_model(f, verbose=False):
                    for b in r.boxes:
                        if float(b.conf[0]) < 0.4: continue
                        x1,y1,x2,y2 = map(int, b.xyxy[0])
                        tl_dets.append({'class':tl_model.names[int(b.cls[0])],'confidence':float(b.conf[0]),
                                       'bbox':{'x1':x1/w,'y1':y1/h,'x2':x2/w,'y2':y2/h}})
                        stats['tl'] += 1
                if tl_dets and sio.connected:
                    max_conf = max(tl_dets, key=lambda x: x['confidence'])
                    sio.emit('traffic_light', {'cameraId':c,'imageId':i,'traffic_status':max_conf['class'],
                             'detections':tl_dets,'created_at':t})
                    log(f"🚦 {c[-4:]}: {max_conf['class']}")
            
            stats['f'] += 1
            time.sleep(0.03)
        except queue.Empty: pass
        except Exception as e: log(f"❌ {e}")

def process_license_plate():
    log("🔧 LP OCR thread started")
    while running:
        try:
            data = lp_queue.get(timeout=0.5)
            if data is None: continue
            cam_id, img_id, violations, buffer, detections = data
            
            img = cv2.imdecode(np.frombuffer(buffer, np.uint8), cv2.IMREAD_COLOR)
            if img is None: continue
            h, w = img.shape[:2]
            
            plates = lp_detector(img, size=1920)
            plate_list = plates.pandas().xyxy[0].values.tolist()
            results = {}
            
            for p in plate_list:
                if p[4] < 0.3: continue
                x1,y1,x2,y2 = map(int, p[:4])
                crop = img[max(0,y1):min(h,y2), max(0,x1):min(w,x2)]
                if crop.size == 0: continue
                
                # Find which vehicle this plate belongs to
                vid = None
                for det in detections:
                    bx1 = det['bbox']['x1']*w
                    by1 = det['bbox']['y1']*h
                    bx2 = det['bbox']['x2']*w
                    by2 = det['bbox']['y2']*h
                    if x1 >= bx1 and x2 <= bx2 and y1 >= by1 and y2 <= by2:
                        vid = det.get('id')
                        break
                if vid is None: continue
                
                # Read plate
                text = read_plate(lp_ocr, crop)
                if text == 'unknown':
                    text = read_plate(lp_ocr, deskew(crop))
                
                # Validate Vietnamese plate format
                if text != 'unknown' and re.match(r'^[0-9]{2}[A-Z]{1,2}[0-9]{1,5}$', text):
                    results[vid] = text
                    stats['lp'] += 1
            
            if results and sio.connected:
                sio.emit('violation_license_plate', {'camera_id':cam_id,'image_id':img_id,
                         'license_plates':results,'violations':violations})
                log(f"🚗 LP: {results}")
                
        except queue.Empty: pass
        except Exception as e: log(f"❌ LP: {e}")

# ---------- Socket Events ----------
@sio.event
def connect():
    stats['c'] += 1
    log(f"✅ CONNECTED #{stats['c']}")
    sio.emit('join_all_camera')

@sio.event
def disconnect():
    stats['dc'] += 1
    log(f"⚠️ DISCONNECTED #{stats['dc']}")

@sio.on('image')
def on_image(d):
    try:
        buf = d.get('buffer') or d.get('image')
        if isinstance(buf, dict): buf = bytes(buf.get('data', []))
        if isinstance(buf, str): buf = base64.b64decode(buf)
        img = Image.open(io.BytesIO(buf))
        f = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        if f is None: return
        
        c = d['cameraId']
        if c not in cam_queues:
            cam_queues[c] = queue.Queue(maxsize=3)
            threading.Thread(target=process_camera, args=(c,), daemon=True).start()
        
        q = cam_queues[c]
        while not q.empty():
            try: q.get_nowait()
            except: break
        q.put((f, c, d['imageId'], d.get('created_at',0), d.get('track_line_y',50)))
    except: pass

@sio.on('violation_detect')
def on_violation(d):
    try:
        if lp_detector is None or lp_ocr is None: return
        buf = d.get('buffer')
        if isinstance(buf, dict): buf = bytes(buf.get('data', []))
        lp_queue.put((d.get('camera_id'), d.get('image_id'), d.get('violations'), buf, d.get('detections', [])))
    except: pass

def initial_connect():
    while running and not sio.connected:
        log(f"🔄 Connecting to {SERVER_URL}...")
        try:
            sio.connect(SERVER_URL, transports=['websocket'])
            break
        except: time.sleep(2)

# Start
log(f"🎯 {SERVER_URL}")
if lp_detector and lp_ocr:
    threading.Thread(target=process_license_plate, daemon=True).start()
threading.Thread(target=initial_connect, daemon=True).start()

try:
    while running:
        time.sleep(60)
        log(f"📊 F:{stats['f']} V:{stats['v']} TL:{stats['tl']} LP:{stats['lp']} C:{stats['c']} DC:{stats['dc']}")
except:
    running = False